## реализация данной статьи https://habr.com/ru/articles/769124/

## Новая версия

In [1]:
%%capture
!pip install accelerate==0.29.2
!pip install bitsandbytes==0.43.1
!pip install langchain
!pip install peft

In [2]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate
import pandas as pd

In [3]:
# Load model from HuggingFace Hub
sent_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
sent_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [5]:
from transformers import AutoModelForCausalLM

peft_model_id = "IlyaGusev/saiga_mistral_7b_lora"
model = AutoModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", load_in_8bit=True)

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_id, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
from transformers import Trainer, TrainingArguments

    
BATCH_SIZE = 4
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 50
OUTPUT_DIR = "/kaggle/working/tmp"

training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        # auto_find_batch_size=True,
        learning_rate=LEARNING_RATE,
        num_train_epochs=TRAIN_STEPS,
#         logging_strategy="steps",
#         logging_dir=f"{output_dir}/logs",
        logging_steps=50
    )


In [33]:
texts = pd.read_csv("/kaggle/input/zulip-wekan/qa_preproc_wekan.csv", index_col=False)
texts["qa"] = texts.apply(lambda row: f"<s>Вопрос: {row['question']}\nОтвет: {row['answer']}</s>", axis=1)

In [96]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=texts["qa"],
    tokenizer=tokenizer
)

peft_model_id = "results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


('results/tokenizer_config.json',
 'results/special_tokens_map.json',
 'results/tokenizer.model',
 'results/added_tokens.json',
 'results/tokenizer.json')

In [89]:
def get_embedding(sentence):
    
    #Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

In [90]:
def get_answer(model, tokenizer, info_prompt, question):
    # Tokenize question
    prompt = info_prompt.format(question=question)
    inputs = tokenizer(prompt, padding=True, truncation=True, max_length=128).input_ids.cuda()
    # Generate output
    
    outputs = model.generate(input_ids=inputs, 
                            max_length=128,  # Adjust the max_length as needed
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    # Decode outputs
    output_decode = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    parsed_answer = output_decode
    return parsed_answer

In [91]:
f = lambda x : ' '.join(x)
info_prompt = PromptTemplate.from_template(f(texts["qa"]))

In [92]:
answers = []
emb_database = torch.empty((0, 384), dtype=torch.float16)

In [93]:
question = "Не зачислены часы в wekan" 
emb = get_embedding(question)

In [94]:
def get_cos_sim(question):
    cos_sim = F.cosine_similarity(emb_database, emb, dim=1, eps=1e-8)
    return cos_sim
  
get_cos_sim(question)

tensor([])

In [95]:
answer = get_answer(model, tokenizer, info_prompt, question)
emb_database = torch.cat((emb_database, emb), 0)
answers.append(answer)
print(f'Answer from model: {answer}')

AttributeError: 'list' object has no attribute 'cuda'

## Старая версия

In [5]:
# adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"

# tokenizer = AutoTokenizer.from_pretrained(
#               adapt_model_name,
#               trust_remote_code=True)

# tokenizer.pad_token = tokenizer.eos_token
# device_map = {"": 0}
# model = AutoPeftModelForCausalLM.from_pretrained(
#             adapt_model_name,
#             load_in_8bit = True,
#             device_map=device_map,
#             torch_dtype=torch.bfloat16
# )

# model = model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

Сохранение скачанной модели, чтобы какждый раз при старте ноутбука не скачивать

In [6]:
OUTPUT_DIR = "/kaggle/working/tmp"
model.save_pretrained(OUTPUT_DIR)

In [37]:
def get_embedding(sentence):
    
    #Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

In [8]:
answers = []
emb_database = torch.empty((0, 384), dtype=torch.float32)

In [9]:
texts = pd.read_csv("/kaggle/input/zulip-wekan/qa_preproc_wekan.csv", index_col=False)
texts["qa"] = texts['question'] + texts['answer']

,question,answer,qa
309,Добрый день! Я из проекта 394. На нашей доске ...,"@Ксения Грязева , добрый день! исправлено",Добрый день! Я из проекта 394. На нашей доске ...
246,"@Влада Леушина , иногда сбрасывание сессии пом...",@Валерия Немна Так из кабинета ссылка введет в...,"@Влада Леушина , иногда сбрасывание сессии пом..."
50,Здравствуйте! Назначьте пожалуйста Американова...,"@Александр Американов , добрый вечер! Назначили.",Здравствуйте! Назначьте пожалуйста Американова...
366,"@Техническая поддержка МИЭМ добрый день, по с...","@Андрей Ощепков Добрый день, приняли Ваш запр...","@Техническая поддержка МИЭМ добрый день, по с..."
120,"@Даниил Колесов , @Александра Шарудилова , @На...","@Валерия Немна , спасибо, доска уже появилась","@Даниил Колесов , @Александра Шарудилова , @На..."


In [10]:
f = lambda x : ' '.join(x)
text = f(texts_cut['qa']) + "{question}\nbot: Вот ответ на ваш вопрос длиной не более 10 слов"
info_prompt_less10 = PromptTemplate.from_template(text)

In [11]:
len(text)

210189

Функция из статьи

In [10]:
# def get_answer(model, info_prompt, question):
    
#     prompt = info_prompt.format(question=question)   
#     inputs = tokenizer(prompt, return_tensors="pt")
#     outputs = model.generate(input_ids=inputs["input_ids"], 
#                             top_p=0.5,
#                             temperature=0.3,
#                             attention_mask=inputs["attention_mask"],
#                             max_new_tokens=100,
#                             pad_token_id=tokenizer.eos_token_id,
#                             do_sample=True)

#     output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
#     parsed_answer = output.split("Вот ответ на ваш вопрос длиной не более 10 слов:")[1].strip()

#     if "bot:" in parsed_answer:
#         parsed_answer = parsed_answer.split("bot:")[0].strip()

#     return parsed_answer

Переписанная функция из статьи

In [ ]:
def get_answer(model, tokenizer, info_prompt, question):
    # Tokenize all questions
    prompt = info_prompt.format(question=question)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).input_ids.cuda()
    print(tokenizer.eos_token_id)
    # Generate output
    with torch.no_grad():
        outputs = model.generate(input_ids=inputs, 
                                max_length=128,  # Adjust the max_length as needed
                                pad_token_id=tokenizer.eos_token_id,
                                do_sample=True)

    # Decode outputs
    output_decode = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     parsed_answer = output_decode.split("Вот ответ на ваш вопрос длиной не более 10 слов:")[1].strip()

#     if "bot:" in parsed_answer:
#         parsed_answer = parsed_answer.split("bot:")[0].strip()
    parsed_answer = output_decode
    return parsed_answer

In [ ]:
question = "Как авторизоваться в векан?" 
emb = get_embedding(question)

In [ ]:
def get_cos_sim(question):
    cos_sim = F.cosine_similarity(emb_database, emb, dim=1, eps=1e-8)
    return cos_sim
  
get_cos_sim(question)

In [ ]:
answer = get_answer(model, tokenizer, info_prompt_less10, question)
emb_database = torch.cat((emb_database, emb), 0)
answers.append(answer)
print(f'Answer from model: {answer}')

In [71]:
questions = [
    "Не зачтены часы в карточке в wekan",
    "Не  отображается доска в wekan",
    "Не зачисляются часы в карточке",
    "Руководитель не может проставить часы",
    "Не найдена доска в wekan",
    "Можно ли удалить неверно зачтенную карточку?",
    "Не получается авторизоваться в wekan"
            ] 

In [73]:
for q in questions:
    print(q)
    emb = get_embedding(q)
#     answer = get_answer(info_prompt_less10, q)
#     emb_database = torch.cat((emb_database, emb), 0)
#     answers.append(answer)
#     print(f'Answer from model: {answer}')
    
    cos_sim = get_cos_sim(q)
    max_value, max_index = torch.max(get_cos_sim(q), dim=0)

    if max_value > 0.9:
        answer = answers[max_index]
        print(f'DATABASE: {answer}')
    else:
        answer = get_answer(model, tokenizer, info_prompt_less10, q)
        emb_database = torch.cat((emb_database, emb), 0)
        answers.append(answer)
        print(f'MODEL: {answer}')
    print()

Не зачтены часы в карточке в wekan
MODEL: Кожакин Кирилл Геннадьевич &lt;kgkozhakin@edu.hse.ru&gt;,
Козлов Александр Сергеевич &lt;askozlov_13@edu.hse.ru&gt;,
Шаповалов Михаил Вадимович &lt;mvshapovalov@edu.hse.ru&gt;@Александр Подшивалов , теперь все участники присоединены к доске. Шаповалов говорит, что они

Не  отображается доска в wekan
MODEL: Кожакин Кирилл Геннадьевич &lt;kgkozhakin@edu.hse.ru&gt;,
Козлов Александр Сергеевич &lt;askozlov_13@edu.hse.ru&gt;,
Шаповалов Михаил Вадимович &lt;mvshapovalov@edu.hse.ru&gt;@Александр Подшивалов , теперь все участники присоединены к доске. Шаповалов говорит, что про

Не зачисляются часы в карточке
MODEL: Кожакин Кирилл Геннадьевич &lt;kgkozhakin@edu.hse.ru&gt;,
Козлов Александр Сергеевич &lt;askozlov_13@edu.hse.ru&gt;,
Шаповалов Михаил Вадимович &lt;mvshapovalov@edu.hse.ru&gt;@Александр Подшивалов , теперь все участники присоединены к доске. Шаповалов говорит, что все

Руководитель не может проставить часы
MODEL: Кожакин Кирилл Геннадьевич 